In [1]:
import os
import datetime
import time

import IPython
import IPython.display
from ipywidgets import IntProgress
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import r2_score
import math

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

C:\Users\iasha\AppData\Local\Temp\ipykernel_27580\424092037.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def convert_line_to_list(line: str):
    split_line = line.split(' ')
    result_list = []

    for current in split_line:
        if current == '\n':
            continue
            
        result_list.append(int(current))

    return result_list

In [4]:
def get_mlt(clustering_unit_index):
    return clustering_unit_index // 80 * 0.25

def get_magneticLat(clustering_unit_index):
    return clustering_unit_index % 80 * 0.5 + 50

In [5]:
def cluster_unit_index_to_file_path(clustering_unit_index):
    mlt = get_mlt(clustering_unit_index)
    magnetic_lat = get_magneticLat(clustering_unit_index)
    
    return f"C:\\Coding\\final-qualifying-work_2023-2024\\OvationPrimeDataCollector\\north-nowcast-diffuse\\{mlt:.2f}_{magnetic_lat:.1f}.csv"

In [6]:
def get_cluster_centre(cluster):
    cluster_centre = pd.read_csv(cluster_unit_index_to_file_path(cluster[0]), sep=';', low_memory=False)

    f = IntProgress(min=0, max=len(cluster))
    display(f)
    i = 0
    for clustering_unit_index in cluster:
        # if i % 100 == 0:
        #     f.value += 100
        f.value += 1
        if i == 0:
            i += 1
            continue
        current_df = pd.read_csv(cluster_unit_index_to_file_path(clustering_unit_index), sep=';', low_memory=False)
        cluster_centre['y'] += current_df['y']
        i += 1

    cluster_centre['y'] = cluster_centre['y'].divide(len(cluster))
    return cluster_centre

In [7]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [8]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [9]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [10]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [11]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')
    
  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [12]:
def investigate(cluster_index, model, conv_width, clusters, save_data):
    cluster_centre = get_cluster_centre(clusters[cluster_index])
    cluster_centre = cluster_centre.drop(['date'], axis=1)
    cluster_centre = cluster_centre.drop(['segment'], axis=1)
    
    n = len(cluster_centre)
    train_df = cluster_centre[0:int(n*0.7)]
    val_df = cluster_centre[int(n*0.7):int(n*0.9)]
    test_df = cluster_centre[int(n*0.9):]

    train_mean = train_df.mean()
    train_std = train_df.std()

    train_df = (train_df - train_mean) / train_std
    val_df = (val_df - train_mean) / train_std
    test_df = (test_df - train_mean) / train_std
    
    conv_window = WindowGenerator(
    input_width=conv_width,
    label_width=1,
    shift=1,
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    label_columns=['y']
    )

    model = keras.models.Sequential.from_config(model.get_config())
    
    history = compile_and_fit(model, conv_window)

    inputs, labels = conv_window.example

    val_df_numpy = val_df.to_numpy()
    train_df_numpy = train_df.to_numpy()
    new_true_values_val = []
    new_true_values_train = []
    new_input_val = []
    new_input_train = []

    new_true_values_val = val_df_numpy[3:].flatten()
    new_input_val = np.column_stack([val_df_numpy[0:-3], val_df_numpy[1:-2], val_df_numpy[2:-1]])

    new_true_values_train = train_df_numpy[3:].flatten()
    new_input_train = np.column_stack([train_df_numpy[0:-3], train_df_numpy[1:-2], train_df_numpy[2:-1]])
        
    val_predictions = model(np.array(new_input_val)).numpy()
    train_predictions = model(np.array(new_input_train)).numpy()
    
    new_val_predictions = val_predictions.flatten()
    new_train_predictions = train_predictions.flatten()
    
    new_true_values_val = (new_true_values_val * train_std['y']) + train_mean['y']
    new_val_predictions = (new_val_predictions * train_std['y']) + train_mean['y']

    new_true_values_train = (new_true_values_train * train_std['y']) + train_mean['y']
    new_train_predictions = (new_train_predictions * train_std['y']) + train_mean['y']

    print("***TRAIN***:")
    output_file.write("***TRAIN***:\n")
    print(f"correlation related to cluster centre: {np.corrcoef(new_true_values_train, new_train_predictions)[0][1]:.2f}")
    output_file.write(f"correlation related to cluster centre: {np.corrcoef(new_true_values_train, new_train_predictions)[0][1]:.2f}\n")
    print(f"r2 score related to cluster centre: {r2_score(new_true_values_train, new_train_predictions):.2f}")
    output_file.write(f"r2 score related to cluster centre: {r2_score(new_true_values_train, new_train_predictions):.2f}\n")
    print("***VALIDATION***:")
    output_file.write("***VALIDATION***:\n")
    print(f"correlation related to cluster centre: {np.corrcoef(new_true_values_val, new_val_predictions)[0][1]:.2f}")
    output_file.write(f"correlation related to cluster centre: {np.corrcoef(new_true_values_val, new_val_predictions)[0][1]:.2f}\n")
    print(f"r2 score related to cluster centre: {r2_score(new_true_values_val, new_val_predictions):.2f}")
    output_file.write(f"r2 score related to cluster centre: {r2_score(new_true_values_val, new_val_predictions):.2f}\n")

    sum_correlation_val = 0
    sum_r2_val = 0

    f = IntProgress(min=0, max=len(clusters[cluster_index]))
    display(f)

    i = 0
    for clustering_unit_index in clusters[cluster_index]:
        if (i % 100 == 0):
            f.value += 100
        
        current_df = pd.read_csv(cluster_unit_index_to_file_path(clustering_unit_index), sep=';', low_memory=False)
        current_df = current_df.drop(['date'], axis=1)
        current_df = current_df.drop(['segment'], axis=1)

        n = len(current_df)
        current_train_df = current_df[0:int(n*0.7)]
        current_val_df = current_df[int(n*0.7):int(n*0.9)]
        current_test_df = current_df[int(n*0.9):]

        current_train_df = (current_train_df - train_mean) / train_std
        current_val_df = (current_val_df - train_mean) / train_std
        current_test_df = (current_test_df - train_mean) / train_std

        current_val_df_numpy = current_val_df.to_numpy()
        current_new_true_values_val = current_val_df_numpy[3:].flatten()
        current_new_input_val = np.column_stack([current_val_df_numpy[0:-3], current_val_df_numpy[1:-2], current_val_df_numpy[2:-1]])

        current_val_predictions = model(np.array(current_new_input_val)).numpy()
        current_new_val_predictions = current_val_predictions.flatten()

        current_new_true_values_val = (current_new_true_values_val * train_std['y']) + train_mean['y']
        current_new_val_predictions = (current_new_val_predictions * train_std['y']) + train_mean['y']
            
        sum_correlation_val += np.corrcoef(current_new_true_values_val, current_new_val_predictions)[0][1]
        sum_r2_val += r2_score(current_new_true_values_val, current_new_val_predictions)

        i += 1

    if (save_data):
        f = open(f"train mean and std for cluster {cluster_index}.txt", "w")
        f.write(str(train_mean.mean()) + ' ' + str(train_std.mean()))
        f.close()
        model.save(f"model for cluster {cluster_index}")

    cluster_len = len(clusters[cluster_index])
    
    return sum_correlation_val / cluster_len, sum_r2_val / cluster_len,

In [13]:
def do_investigations(model, clusters, conv_width):
    cluster_sizes = []
    correlations = []
    r2_scores = []

    for i in range(len(clusters)):
        print(f"---do_investigations. cluster index : {i}---")
        output_file.write(f"---do_investigations. cluster index : {i}---\n")
        average_correlation, average_r2 = investigate(i, model, conv_width, clusters, False)
        
        print("***VALIDATION***:")
        output_file.write("***VALIDATION***:\n")
        print(f"average correlation: {average_correlation:.2f}")
        output_file.write(f"average correlation: {average_correlation:.2f}\n")
        print(f"average r2: {average_r2:.2f}")
        output_file.write(f"average r2: {average_r2:.2f}\n")

        cluster_sizes.append(len(clusters[i]))
        correlations.append(average_correlation)
        r2_scores.append(average_r2)

    return cluster_sizes, correlations, r2_scores

In [14]:
def print_metrics(cluster_sizes, correlations, r2_scores, clusters):
    total_size = 0
    correlation_sum = 0
    r2_score_sum = 0
    
    bad_clusters_indices = []

    for i in range(len(cluster_sizes)):
        if (math.isnan(correlations[i]) or (r2_scores[i] < 0)):
            bad_clusters_indices.append(i)
            continue
        
        correlation_sum += cluster_sizes[i] * correlations[i]
        r2_score_sum += cluster_sizes[i] * r2_scores[i]
        total_size += cluster_sizes[i]

    weighted_average_correlation = correlation_sum / total_size
    weighted_average_r2_score = r2_score_sum / total_size

    print(f"weighted_average_correlation: {weighted_average_correlation:.2f}")
    output_file.write(f"weighted_average_correlation: {weighted_average_correlation:.2f}\n")
    print(f"weighted_average_r2_score: {weighted_average_r2_score:.2f}")
    output_file.write(f"weighted_average_r2_score: {weighted_average_r2_score:.2f}\n")

    bad_clusters_size_sum = 0
    total_clusters_size_sum = 0

    for cluster_size in cluster_sizes:
        total_clusters_size_sum += cluster_size
    
    for bad_cluster_index in bad_clusters_indices:
        bad_clusters_size_sum += len(clusters[bad_cluster_index])

    bad_clusters_rate = bad_clusters_size_sum / total_clusters_size_sum

    print(f"bad clusters rate: {bad_clusters_rate:.2f}%")
    output_file.write(f"bad clusters rate: {bad_clusters_rate:.2f}%\n")

In [15]:
def show_results(model, clusters, conv_width):
    cluster_sizes, correlations, r2_scores = do_investigations(model, clusters, conv_width)
    print_metrics(cluster_sizes, correlations, r2_scores, clusters)

In [16]:
CONV_WIDTH = 3

In [17]:
multi_step_dense_model = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

In [18]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.LSTM(128, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [19]:
rnn_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [20]:
def get_clusters(path):
    file = open(path, 'r')
    lines = file.readlines()
    file.close()

    clusters = []

    for line in lines:
        if line.strip() == '':
            continue
        clusters.append(convert_line_to_list(line))

    return clusters

In [21]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = first_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
msd mst2 5.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5195)

Epoch 1/20

368/368 [==============================] - 3s 4ms/step - loss: 0.2494 - mean_absolute_error: 0.3099 - val_loss: 0.2198 - val_mean_absolute_error: 0.2764
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2070 - mean_absolute_error: 0.2848 - val_loss: 0.2128 - val_mean_absolute_error: 0.2818
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2058 - mean_absolute_error: 0.2835 - val_loss: 0.2163 - val_mean_absolute_error: 0.2834
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2042 - mean_absolute_error: 0.2831 - val_loss: 0.2192 - val_mean_absolute_error: 0.2866
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.80
***VALIDATION***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.81


IntProgress(value=0, max=5195)

***VALIDATION***:
average correlation: nan
average r2: -23711893202059195031750705152.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 24---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2038 - mean_absolute_error: 0.3101 - val_loss: 0.1842 - val_mean_absolute_error: 0.3101
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1675 - mean_absolute_error: 0.2811 - val_loss: 0.1787 - val_mean_absolute_error: 0.2957
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1664 - mean_absolute_error: 0.2799 - val_loss: 0.1794 - val_mean_absolute_error: 0.3039
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1664 - mean_absolute_error: 0.2799 - val_loss: 0.1775 - val_mean_absolute_error: 0.2981
Epoch 5/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1662 - mean_absolute_error: 0.2792 - val_loss: 0.1828 - val_mean_absolute_error: 0.3119
Epoch 6/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1658 - mean_absolute_error: 0.2790 - val_loss: 0.1772 - val_mean_absolute_error: 0.2967
Epoch 7/20
368/3

IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.22
bad clusters rate: 0.68%
------------------------------------------------------


In [22]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = second_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
msd mst2 7.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5323)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.3754 - mean_absolute_error: 0.3617 - val_loss: 0.2018 - val_mean_absolute_error: 0.2811
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2070 - mean_absolute_error: 0.2892 - val_loss: 0.1980 - val_mean_absolute_error: 0.2773
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2073 - mean_absolute_error: 0.2891 - val_loss: 0.2059 - val_mean_absolute_error: 0.2827
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2061 - mean_absolute_error: 0.2877 - val_loss: 0.1998 - val_mean_absolute_error: 0.2869
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.80
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=5323)

***VALIDATION***:
average correlation: nan
average r2: -77560186374940242574162526208.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.92
average r2: 0.85
---do_investigations. cluster index : 9---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2335 - mean_absolute_error: 0.3318 - val_loss: 0.1859 - val_mean_absolute_error: 0.2969
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1660 - mean_absolute_error: 0.2790 - val_loss: 0.1820 - val_mean_absolute_error: 0.2910
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1664 - mean_absolute_error: 0.2796 - val_loss: 0.1843 - val_mean_absolute_error: 0.2918
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1660 - mean_absolute_error: 0.2791 - val_loss: 0.1850 - val_mean_absolute_error: 0.2931
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.83
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.91
weighted_average_r2_score: 0.80
bad clusters rate: 0.97%
------------------------------------------------------


In [23]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = third_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
msd mst2 5 - 0.5.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=122)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2435 - mean_absolute_error: 0.3348 - val_loss: 0.1746 - val_mean_absolute_error: 0.2817
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2066 - mean_absolute_error: 0.2972 - val_loss: 0.1739 - val_mean_absolute_error: 0.2889
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2061 - mean_absolute_error: 0.2968 - val_loss: 0.1721 - val_mean_absolute_error: 0.2798
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2053 - mean_absolute_error: 0.2956 - val_loss: 0.1722 - val_mean_absolute_error: 0.2832
Epoch 5/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2050 - mean_absolute_error: 0.2956 - val_loss: 0.1729 - val_mean_absolute_error: 0.2803
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to 

IntProgress(value=0, max=122)

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 113---


IntProgress(value=0, max=10)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2934 - mean_absolute_error: 0.3848 - val_loss: 0.1844 - val_mean_absolute_error: 0.3037
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2161 - mean_absolute_error: 0.3264 - val_loss: 0.1817 - val_mean_absolute_error: 0.2965
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2153 - mean_absolute_error: 0.3255 - val_loss: 0.1917 - val_mean_absolute_error: 0.3072
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2145 - mean_absolute_error: 0.3246 - val_loss: 0.1919 - val_mean_absolute_error: 0.3116
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.78
***VALIDATION***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.80


IntProgress(value=0, max=10)

***VALIDATION***:
average correlation: 0.90
average r2: 0.80
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.81
bad clusters rate: 0.88%
------------------------------------------------------


In [24]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = fourth_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
msd mst2 5 - 1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=122)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2944 - mean_absolute_error: 0.3392 - val_loss: 0.1764 - val_mean_absolute_error: 0.2953
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2071 - mean_absolute_error: 0.2978 - val_loss: 0.1741 - val_mean_absolute_error: 0.2940
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2071 - mean_absolute_error: 0.2975 - val_loss: 0.1766 - val_mean_absolute_error: 0.2960
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2066 - mean_absolute_error: 0.2974 - val_loss: 0.1716 - val_mean_absolute_error: 0.2800
Epoch 5/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2053 - mean_absolute_error: 0.2956 - val_loss: 0.1720 - val_mean_absolute_error: 0.2870
Epoch 6/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2053 - mean_absolute_error: 0.2964 - val_loss: 0.1716 - val_mean_absolute_error: 0.2857
Epoch 7/20
368/3

IntProgress(value=0, max=122)

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
---do_investigations. cluster index : 39---


IntProgress(value=0, max=3)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2388 - mean_absolute_error: 0.3127 - val_loss: 0.1570 - val_mean_absolute_error: 0.2514
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2085 - mean_absolute_error: 0.2783 - val_loss: 0.1583 - val_mean_absolute_error: 0.2456
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2072 - mean_absolute_error: 0.2766 - val_loss: 0.1580 - val_mean_absolute_error: 0.2511
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.80
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=3)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.91
weighted_average_r2_score: 0.81
bad clusters rate: 0.90%
------------------------------------------------------


In [21]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = first_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"lstm {os.path.split(selected_path)[-1]}")
output_file.write(f"lstm {os.path.split(selected_path)[-1]}\n")
show_results(lstm_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
lstm mst2 5.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5195)

Epoch 1/20

368/368 [==============================] - 5s 6ms/step - loss: 0.2765 - mean_absolute_error: 0.3375 - val_loss: 0.2252 - val_mean_absolute_error: 0.2938
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2071 - mean_absolute_error: 0.2871 - val_loss: 0.2167 - val_mean_absolute_error: 0.2780
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2065 - mean_absolute_error: 0.2859 - val_loss: 0.2173 - val_mean_absolute_error: 0.2852
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2048 - mean_absolute_error: 0.2851 - val_loss: 0.2155 - val_mean_absolute_error: 0.2815
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2042 - mean_absolute_error: 0.2845 - val_loss: 0.2201 - val_mean_absolute_error: 0.2842
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2041 - mean_absolute_error: 0.2845 - val_loss: 0.2212 - val_mean_absolute_error: 0.2832
***TRAIN***:
co

IntProgress(value=0, max=5195)

***VALIDATION***:
average correlation: nan
average r2: -16467652573421146643743375360.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 24---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 4s 6ms/step - loss: 0.2352 - mean_absolute_error: 0.3354 - val_loss: 0.1814 - val_mean_absolute_error: 0.3023
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1666 - mean_absolute_error: 0.2809 - val_loss: 0.1773 - val_mean_absolute_error: 0.2940
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1661 - mean_absolute_error: 0.2797 - val_loss: 0.1776 - val_mean_absolute_error: 0.2956
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1653 - mean_absolute_error: 0.2791 - val_loss: 0.1773 - val_mean_absolute_error: 0.2939
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.84
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.91
weighted_average_r2_score: 0.82
bad clusters rate: 0.94%
------------------------------------------------------


In [22]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = second_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"lstm {os.path.split(selected_path)[-1]}")
output_file.write(f"lstm {os.path.split(selected_path)[-1]}\n")
show_results(lstm_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
lstm mst2 7.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5323)

Epoch 1/20
368/368 [==============================] - 4s 6ms/step - loss: 0.2793 - mean_absolute_error: 0.3379 - val_loss: 0.2010 - val_mean_absolute_error: 0.2860
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2078 - mean_absolute_error: 0.2908 - val_loss: 0.2016 - val_mean_absolute_error: 0.2826
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2074 - mean_absolute_error: 0.2909 - val_loss: 0.2012 - val_mean_absolute_error: 0.2788
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.80
***VALIDATION***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.81


IntProgress(value=0, max=5323)

***VALIDATION***:
average correlation: nan
average r2: -41389384363560504213960130560.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.92
average r2: 0.85
---do_investigations. cluster index : 9---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 5s 6ms/step - loss: 0.2358 - mean_absolute_error: 0.3360 - val_loss: 0.1809 - val_mean_absolute_error: 0.3050
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1664 - mean_absolute_error: 0.2799 - val_loss: 0.1826 - val_mean_absolute_error: 0.3086
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1659 - mean_absolute_error: 0.2791 - val_loss: 0.1826 - val_mean_absolute_error: 0.3114
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.83
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.91
weighted_average_r2_score: 0.81
bad clusters rate: 0.97%
------------------------------------------------------


In [23]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = third_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"lstm {os.path.split(selected_path)[-1]}")
output_file.write(f"lstm {os.path.split(selected_path)[-1]}\n")
show_results(lstm_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
lstm mst2 5 - 0.5.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=122)

Epoch 1/20
368/368 [==============================] - 5s 6ms/step - loss: 0.2861 - mean_absolute_error: 0.3484 - val_loss: 0.1732 - val_mean_absolute_error: 0.2787
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2086 - mean_absolute_error: 0.2997 - val_loss: 0.1727 - val_mean_absolute_error: 0.2835
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2073 - mean_absolute_error: 0.2984 - val_loss: 0.1739 - val_mean_absolute_error: 0.2816
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2075 - mean_absolute_error: 0.2984 - val_loss: 0.1713 - val_mean_absolute_error: 0.2741
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2073 - mean_absolute_error: 0.2974 - val_loss: 0.1720 - val_mean_absolute_error: 0.2752
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2061 - mean_absolute_error: 0.2970 - val_loss: 0.1720 - val_mean_absolute_error: 0.2748
***TRAIN***:
cor

IntProgress(value=0, max=122)

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 113---


IntProgress(value=0, max=10)

Epoch 1/20
368/368 [==============================] - 5s 6ms/step - loss: 0.2659 - mean_absolute_error: 0.3666 - val_loss: 0.1861 - val_mean_absolute_error: 0.3021
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2165 - mean_absolute_error: 0.3267 - val_loss: 0.1792 - val_mean_absolute_error: 0.2930
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2154 - mean_absolute_error: 0.3253 - val_loss: 0.1759 - val_mean_absolute_error: 0.2942
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2149 - mean_absolute_error: 0.3247 - val_loss: 0.1758 - val_mean_absolute_error: 0.2931
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2147 - mean_absolute_error: 0.3247 - val_loss: 0.1781 - val_mean_absolute_error: 0.2923
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2147 - mean_absolute_error: 0.3246 - val_loss: 0.1810 - val_mean_absolute_error: 0.3016
***TRAIN***:
cor

IntProgress(value=0, max=10)

***VALIDATION***:
average correlation: 0.91
average r2: 0.81
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.81
bad clusters rate: 0.88%
------------------------------------------------------


In [24]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = fourth_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"lstm {os.path.split(selected_path)[-1]}")
output_file.write(f"lstm {os.path.split(selected_path)[-1]}\n")
show_results(lstm_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
lstm mst2 5 - 1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=122)

Epoch 1/20
368/368 [==============================] - 5s 7ms/step - loss: 0.2669 - mean_absolute_error: 0.3528 - val_loss: 0.1738 - val_mean_absolute_error: 0.2879
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2098 - mean_absolute_error: 0.3006 - val_loss: 0.1849 - val_mean_absolute_error: 0.3022
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2084 - mean_absolute_error: 0.2995 - val_loss: 0.1782 - val_mean_absolute_error: 0.2883
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.81


IntProgress(value=0, max=122)

***VALIDATION***:
average correlation: 0.90
average r2: 0.80
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.91
average r2: 0.81
---do_investigations. cluster index : 39---


IntProgress(value=0, max=3)

Epoch 1/20
368/368 [==============================] - 5s 7ms/step - loss: 0.2787 - mean_absolute_error: 0.3262 - val_loss: 0.1580 - val_mean_absolute_error: 0.2515
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2113 - mean_absolute_error: 0.2811 - val_loss: 0.1571 - val_mean_absolute_error: 0.2471
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2131 - mean_absolute_error: 0.2819 - val_loss: 0.1559 - val_mean_absolute_error: 0.2499
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2102 - mean_absolute_error: 0.2796 - val_loss: 0.1565 - val_mean_absolute_error: 0.2455
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2084 - mean_absolute_error: 0.2779 - val_loss: 0.1573 - val_mean_absolute_error: 0.2435
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to 

IntProgress(value=0, max=3)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.81
bad clusters rate: 0.90%
------------------------------------------------------


In [21]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = first_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"rnn {os.path.split(selected_path)[-1]}")
output_file.write(f"rnn {os.path.split(selected_path)[-1]}\n")
show_results(rnn_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
rnn mst2 5.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5195)

Epoch 1/20

368/368 [==============================] - 4s 4ms/step - loss: 0.2379 - mean_absolute_error: 0.3091 - val_loss: 0.2391 - val_mean_absolute_error: 0.2918
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2107 - mean_absolute_error: 0.2870 - val_loss: 0.2329 - val_mean_absolute_error: 0.2957
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2076 - mean_absolute_error: 0.2851 - val_loss: 0.2319 - val_mean_absolute_error: 0.3009
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2079 - mean_absolute_error: 0.2850 - val_loss: 0.2401 - val_mean_absolute_error: 0.3030
Epoch 5/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2087 - mean_absolute_error: 0.2854 - val_loss: 0.2337 - val_mean_absolute_error: 0.3084
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.90
r2 score related to

IntProgress(value=0, max=5195)

***VALIDATION***:
average correlation: nan
average r2: -56062083893136332063833587712.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 24---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 3s 4ms/step - loss: 0.1898 - mean_absolute_error: 0.3026 - val_loss: 0.1796 - val_mean_absolute_error: 0.2923
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1682 - mean_absolute_error: 0.2827 - val_loss: 0.1830 - val_mean_absolute_error: 0.2998
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1672 - mean_absolute_error: 0.2821 - val_loss: 0.1791 - val_mean_absolute_error: 0.2995
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1673 - mean_absolute_error: 0.2815 - val_loss: 0.1803 - val_mean_absolute_error: 0.2965
Epoch 5/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1671 - mean_absolute_error: 0.2817 - val_loss: 0.1793 - val_mean_absolute_error: 0.3051
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.83
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to 

IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.91
weighted_average_r2_score: 0.81
bad clusters rate: 0.94%
------------------------------------------------------


In [22]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = second_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"rnn {os.path.split(selected_path)[-1]}")
output_file.write(f"rnn {os.path.split(selected_path)[-1]}\n")
show_results(rnn_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
rnn mst2 7.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5323)

Epoch 1/20
368/368 [==============================] - 3s 5ms/step - loss: 0.2389 - mean_absolute_error: 0.3111 - val_loss: 0.2121 - val_mean_absolute_error: 0.3006
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2093 - mean_absolute_error: 0.2913 - val_loss: 0.2065 - val_mean_absolute_error: 0.2853
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2084 - mean_absolute_error: 0.2893 - val_loss: 0.2359 - val_mean_absolute_error: 0.3162
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2089 - mean_absolute_error: 0.2917 - val_loss: 0.2147 - val_mean_absolute_error: 0.2877
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.80


IntProgress(value=0, max=5323)

***VALIDATION***:
average correlation: nan
average r2: -36886226446068186591562563584.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.92
average r2: 0.85
---do_investigations. cluster index : 9---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 3s 4ms/step - loss: 0.1956 - mean_absolute_error: 0.3045 - val_loss: 0.1804 - val_mean_absolute_error: 0.2983
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1681 - mean_absolute_error: 0.2817 - val_loss: 0.1822 - val_mean_absolute_error: 0.3011
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1669 - mean_absolute_error: 0.2800 - val_loss: 0.1792 - val_mean_absolute_error: 0.2997
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1667 - mean_absolute_error: 0.2804 - val_loss: 0.1784 - val_mean_absolute_error: 0.2892
Epoch 5/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1669 - mean_absolute_error: 0.2806 - val_loss: 0.1812 - val_mean_absolute_error: 0.2983
Epoch 6/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1661 - mean_absolute_error: 0.2794 - val_loss: 0.1816 - val_mean_absolute_error: 0.3049
***TRAIN***:
cor

IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.81
bad clusters rate: 0.97%
------------------------------------------------------


In [23]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = third_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"rnn {os.path.split(selected_path)[-1]}")
output_file.write(f"rnn {os.path.split(selected_path)[-1]}\n")
show_results(rnn_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
rnn mst2 5 - 0.5.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=122)

Epoch 1/20
368/368 [==============================] - 3s 5ms/step - loss: 0.2443 - mean_absolute_error: 0.3269 - val_loss: 0.1773 - val_mean_absolute_error: 0.2922
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2112 - mean_absolute_error: 0.3018 - val_loss: 0.1793 - val_mean_absolute_error: 0.2911
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2104 - mean_absolute_error: 0.3008 - val_loss: 0.1724 - val_mean_absolute_error: 0.2764
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2082 - mean_absolute_error: 0.2987 - val_loss: 0.1732 - val_mean_absolute_error: 0.2870
Epoch 5/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2098 - mean_absolute_error: 0.2996 - val_loss: 0.1882 - val_mean_absolute_error: 0.2871
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to 

IntProgress(value=0, max=122)

***VALIDATION***:
average correlation: 0.90
average r2: 0.79
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 113---


IntProgress(value=0, max=10)

Epoch 1/20
368/368 [==============================] - 3s 4ms/step - loss: 0.2517 - mean_absolute_error: 0.3548 - val_loss: 0.1768 - val_mean_absolute_error: 0.2927
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2196 - mean_absolute_error: 0.3298 - val_loss: 0.1774 - val_mean_absolute_error: 0.2990
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2173 - mean_absolute_error: 0.3280 - val_loss: 0.1775 - val_mean_absolute_error: 0.2909
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.78
***VALIDATION***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=10)

***VALIDATION***:
average correlation: 0.90
average r2: 0.82
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.80
bad clusters rate: 0.88%
------------------------------------------------------


In [21]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 7.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 0.5.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\mst2 5 - 1.txt"

output_file = open("computational-experiments-output-mst-2.txt", "a")

selected_path = fourth_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"rnn {os.path.split(selected_path)[-1]}")
output_file.write(f"rnn {os.path.split(selected_path)[-1]}\n")
show_results(rnn_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
rnn mst2 5 - 1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=122)

Epoch 1/20

368/368 [==============================] - 4s 4ms/step - loss: 0.2321 - mean_absolute_error: 0.3215 - val_loss: 0.1743 - val_mean_absolute_error: 0.2814
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2106 - mean_absolute_error: 0.3016 - val_loss: 0.1728 - val_mean_absolute_error: 0.2782
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2090 - mean_absolute_error: 0.2999 - val_loss: 0.1745 - val_mean_absolute_error: 0.2920
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2099 - mean_absolute_error: 0.2997 - val_loss: 0.1768 - val_mean_absolute_error: 0.2894
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.81


IntProgress(value=0, max=122)

***VALIDATION***:
average correlation: 0.90
average r2: 0.80
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 39---


IntProgress(value=0, max=3)

Epoch 1/20
368/368 [==============================] - 3s 4ms/step - loss: 0.2412 - mean_absolute_error: 0.3044 - val_loss: 0.1604 - val_mean_absolute_error: 0.2420
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2133 - mean_absolute_error: 0.2814 - val_loss: 0.1652 - val_mean_absolute_error: 0.2619
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2110 - mean_absolute_error: 0.2800 - val_loss: 0.1653 - val_mean_absolute_error: 0.2548
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.81


IntProgress(value=0, max=3)

***VALIDATION***:
average correlation: 0.91
average r2: 0.81
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.81
bad clusters rate: 0.90%
------------------------------------------------------
